# Kaggle Competition: Patfinder Pawpularity score

## 5. Inference code

Import libraries

In [28]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
import timm
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

Device

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


Load test data

Path names: images and model

In [15]:
path = '../input/petfinder-pawpularity-score/'
img_dir = path + 'test/'
path_model = '../models/effnetb0meta_bs32_lr00001_m09.pt'

Load in dataframe

In [16]:
test_data = pd.read_csv(path+'test.csv')
test_data['path'] = path + 'test/' + test_data['Id'] + '.jpg'

print('Samples test:', len(test_data))
test_data.head()

Samples test: 8


,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...
1,43a2262d7738e3d420d453815151079e,0,1,0,0,0,0,1,1,0,0,0,0,../input/petfinder-pawpularity-score/test/43a2...
2,4e429cead1848a298432a0acad014c9d,0,0,0,1,0,1,1,1,0,1,1,1,../input/petfinder-pawpularity-score/test/4e42...
3,80bc3ccafcc51b66303c2c263aa38486,1,0,1,0,0,0,0,0,0,0,1,0,../input/petfinder-pawpularity-score/test/80bc...
4,8f49844c382931444e68dffbe20228f4,1,1,1,0,1,1,0,1,0,1,1,0,../input/petfinder-pawpularity-score/test/8f49...


Load in dataloader

In [33]:
img_size = 224
batch_size = 256
num_workers=0#4
pin_memory=False#True

# To make transformations of the images
transform = transforms.Compose(
                   [
                    transforms.ToTensor(),
                    transforms.Resize((img_size,img_size)),
                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])


class Dataset(torch.utils.data.Dataset):
    def __init__(self, img_dir, label_dataframe, img_name, meta_x, transform=None):
        self.img_dir = img_dir
        self.img_label = label_dataframe
        self.transform = transform
        self.img_name = img_name
        self.meta_x = meta_x
        
    def __len__(self):
        return len(self.img_label)

    def __getitem__(self, ix):
        img_id = self.img_label[self.img_name][ix] 
        meta_x_ix = self.img_label[self.meta_x].iloc[ix]
        meta_x_ix = torch.tensor(meta_x_ix, dtype=torch.float32)
        img_path = self.img_dir + self.img_label[self.img_name][ix] + '.jpg'
        X_ix = plt.imread(img_path)
        if self.transform:
            X_ix = self.transform(X_ix)
        return img_id, (X_ix, meta_x_ix)


meta_x_columns = [
    'Subject Focus','Eyes','Face','Near','Action','Accessory', 
    'Group', 'Collage','Human','Occlusion','Info','Blur'
    ]

dataset_test = Dataset(img_dir, test_data, 'Id', meta_x_columns, transform=transform)
dataloader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, num_workers=num_workers, pin_memory=pin_memory)


In [34]:
for i, (img_id, batch_X) in enumerate(dataloader_test):
    print(f"Batch #{i+1} of size -> image_data: {batch_X[0].shape} and meta_data: {batch_X[1].shape}")

Batch #1 of size -> image_data: torch.Size([8, 3, 224, 224]) and meta_data: torch.Size([8, 12])


Load model

In [19]:
class Tailored_Net(nn.Module):
    def __init__(
        self, timm_model_name, len_meta_x, target_size, dropout_rate=0, pretrained=True, freezepretrained=False):
        super().__init__()
        self.efficientmodel = timm.create_model(timm_model_name, pretrained=pretrained)
        self.n_features = self.efficientmodel.classifier.in_features
        self.efficientmodel.classifier = nn.Identity()
        self.batchnorm = nn.BatchNorm1d(num_features=self.n_features)
        self.fc1 = nn.Linear(self.n_features + len_meta_x, 64)
        self.fc2 = nn.Linear(64, target_size)
        self.dropout = nn.Dropout(dropout_rate)
        # freeze weights of efficientnet
        if freezepretrained:
            for param in self.efficientmodel.parameters():
                param.requires_grad = False

    def forward(self, image, meta_x):
        # efficient net
        x = self.efficientmodel(image)
        # flatten x
        x = x.view(x.size(0), -1)
        # batchnorm
        x = self.batchnorm(x)
        # concat features
        x = torch.cat( (x, meta_x) , dim=1 )
        # fully connected layer + ReLu + Dropout
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout(x)
        # fully connected layer + ReLu
        x = self.fc2(x)
        x = F.relu(x)
        # drop dimension of size 1 (same as .view(-1)) (if not, broadcasting problems with y ground truth)
        x = x.squeeze()
        return x

MODEL_NAME='tf_efficientnet_b0_ns'
model = Tailored_Net(timm_model_name=MODEL_NAME, len_meta_x=12, target_size=1, dropout_rate=0.3, pretrained=True, freezepretrained=True)
model.to(device)
model.load_state_dict(torch.load(path_model))
model.eval()

Tailored_Net(
  (efficientmodel): EfficientNet(
    (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, 

In [31]:
tensordeprueba = torch.load('../others/tensordeprueba.pt', map_location=torch.device(device))
metadatadeprueba = torch.rand(5, 12).to(device)
y_hat = model(tensordeprueba, metadatadeprueba)
y_hat, y_hat.shape

(tensor([36.6466, 41.3709, 39.5213, 31.2207, 37.3744], device='cuda:0',
        grad_fn=<SqueezeBackward0>),
 torch.Size([5]))

Predictions

In [39]:
# make predictions on test set
id_img_all = []
predictions = []
for i, (img_id , (batch_X, batch_x_meta)) in enumerate(dataloader_test):
    print(f"Batch #{i+1} of size -> image_data: {batch_X.shape} and meta_data: {batch_x_meta.shape}")
    y_hat = model(batch_X.to(device), batch_x_meta.to(device))
    predictions.append(y_hat)
    id_img_all.append(img_id)

# concat tensors/lists along batch dimension
predictions = torch.cat(predictions, dim=0)
id_img_all = [item for t in id_img_all for item in t]
#to numpy
predictions = predictions.detach().to("cpu").numpy()
predictions, predictions.shape

Batch #1 of size -> image_data: torch.Size([8, 3, 224, 224]) and meta_data: torch.Size([8, 12])


(array([40.504253, 34.775787, 41.446697, 36.61331 , 42.93597 , 43.08541 ,
        36.395325, 40.659534], dtype=float32),
 (8,))

Save prediction

In [43]:
# save predictions to csv named 'submission.csv'
submission = pd.DataFrame({'Id': id_img_all, 'Pawpularity': predictions})
submission.to_csv('submission.csv', index=False)
submission

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,40.504253
1,43a2262d7738e3d420d453815151079e,34.775787
2,4e429cead1848a298432a0acad014c9d,41.446697
3,80bc3ccafcc51b66303c2c263aa38486,36.613312
4,8f49844c382931444e68dffbe20228f4,42.935970
5,b03f7041962238a7c9d6537e22f9b017,43.085411
6,c978013571258ed6d4637f6e8cc9d6a3,36.395325
7,e0de453c1bffc20c22b072b34b54e50f,40.659534
